In [1]:
import pandas as pd 
import time
import traceback

from memory_profiler import memory_usage

from ast import literal_eval
import numpy as np
from datetime import datetime
from pymongo import MongoClient

import AllSample as ass
import Filter2 as fil2
import TopWords as top
import WordCount3 as woc3
import WordTF_IDF as tf_idf

from flask import Flask, render_template, request, jsonify

In [2]:
t0 = time.time()
client = MongoClient()
db = client.test_database
collection = db.test_collection

reviewsMongo = db.reviews_mini
restaurantsMongo = db.restaurants_change
restaurantsMongoA = db.restaurants_change_a
restaurantsMongoB = db.restaurants_change_b
reviews_plainMongo = db.reviews_plain

allDataLem = pd.DataFrame(list(reviewsMongo.find({})))
t1 = time.time()
print("Database converted in: " + str((t1-t0)/60))
del allDataLem["_id"]
allDataLem = allDataLem.astype({'stars': np.int8})

t2 = time.time()
print("allDataLem done in: " + str((t2-t0)/60))

restaurantNames = pd.DataFrame(list(restaurantsMongo.find({})))
del restaurantNames["_id"]

restaurantNamesA = pd.DataFrame(list(restaurantsMongoA.find({})))
del restaurantNamesA["_id"]
#print(restaurantNamesA)
    
restaurantNamesB = pd.DataFrame(list(restaurantsMongoB.find({})))
del restaurantNamesB["_id"]

restaurantNames = restaurantNames.to_json(orient='records')
restaurantNamesA = restaurantNamesA.to_json(orient='records')
restaurantNamesB = restaurantNamesB.to_json(orient='records')

t3 = time.time()
print("Restaurants done in: " + str((t3-t2)/60))

reviewsPlain = pd.DataFrame(list(reviews_plainMongo.find({})))
del reviewsPlain["_id"]
reviewsPlain = reviewsPlain.astype({'stars': np.int8})
  
t4 = time.time()
print("Just reviews in: " + str((t4-t3)/60))

print("All done in: " + str((t4-t0)/60))

Database converted in: 1.5327070514361063
allDataLem done in: 1.6577715635299684
Restaurants done in: 0.012436870733896892
Just reviews in: 1.1627381523450215
All done in: 2.8329465866088865


In [ ]:
lastDF = pd.DataFrame(index = ['A'], columns = ['new_name'])
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("/index2.html")
    
@app.route("/AllSample")
def allSample():
    return render_template("AllSample2.html")

@app.route("/AllSample2")
def allSample2():
    return render_template("AllSample3.html")

@app.route("/restaurantsNames", methods=['POST'])
def restaurantsNames():
    #Change here and the bellow line!!!
    return jsonify(restaurantNames)

@app.route("/restaurantsNames2", methods=['POST'])
def restaurantsNames2():
    #Change here and the above line!!!
    return jsonify(restaurantNamesA)

@app.route("/topWords", methods=['GET', 'POST'])
def endpointTW():    
    if request.method == 'POST':
        t0 = time.time()
        try:
            currentDF = allDataLem.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            filtered = fil2.main(currentDF, "", newName, rating, dateLow, dateUp)            
            if (filtered.empty):
                return "No data for those filters."        
            status = top.main(filtered["lemmas"])
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return str(status)
    else:
        return "Send a proper POST request!"
    
@app.route("/tf_idf", methods=['GET', 'POST'])
def endpointTF_IDF():    
    if request.method == 'POST':
        try:
            currentDF = allDataLem.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            granulation = request.form['granulation']
            if granulation == 'Year':
                currentDF = fil2.main(currentDF, "", newName, "", "", "")
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", "", dateLow, dateUp)            
                if (filtered1.empty):
                    return "No data for those time periods."         
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
            elif granulation == 'Month':
                currentDF = fil2.main(currentDF, "", newName, "", dateLow[0:5] + '01-01', dateUp[0:5] + '12-31')
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", "", dateLow, dateUp)            
                if (filtered1.empty):
                    return "No data for those time periods."            
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
            elif granulation == 'Rating':
                currentDF = fil2.main(currentDF, "", newName, "", dateLow, dateUp)
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", rating, "", "")            
                if (filtered1.empty):
                    return "No data for those ratings."         
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
            if granulation == 'YearPlus':
                currentDF = fil2.main(currentDF, "", newName, "", "", "")
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", rating, dateLow, dateUp)            
                if (filtered1.empty):
                    return "No data for those time periods."         
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
            elif granulation == 'MonthPlus':
                currentDF = fil2.main(currentDF, "", newName, "", dateLow[0:5] + '01-01', dateUp[0:5] + '12-31')
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", rating, dateLow, dateUp)            
                if (filtered1.empty):
                    return "No data for those time periods."            
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return str(status)
    else:
        return "Send a proper POST request!"
    
@app.route("/yearView")
def endpointYV():    
    return  render_template("/SingleRestaurant2.html")

@app.route("/allRestReviews")
def endpointARR():    
    return  render_template("/SingleRestaurant3.html")

@app.route("/plainReviews", methods=['GET', 'POST'])
def endpointRP():    
    if request.method == 'POST':
        t0 = time.time()
        try:
            plainCDF = reviewsPlain.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            searchWord = request.form['searchWord']
            filtered = fil2.main(plainCDF, "", newName, rating, dateLow, dateUp)            
            if (filtered.empty):
                return "No data for those filters." 
            if (searchWord != ""):
                filtered = filtered[filtered['text'].str.contains(searchWord)]
                if (filtered.empty):
                    return "Search word not found. The search is case sensitive!"
            status = list()
            if filtered.shape[0] >= 5:
                randomRow = np.random.choice(range(filtered.shape[0]), 5, replace=False)
                for i in randomRow:      
                    status.append(str(filtered["stars"].iloc[i]))
                    status.append(str(filtered["date"].iloc[i]))
                    status.append(str(filtered["text"].iloc[i]))
            else:
                for i in range(filtered.shape[0]):
                    status.append(str(filtered["stars"].iloc[i]))
                    status.append(str(filtered["date"].iloc[i]))
                    status.append(str(filtered["text"].iloc[i]))
        except:
            return "Python  error: " + traceback.format_exc().splitlines()[-1]
        return jsonify(status)
    else:
        return "Send a proper POST request!"

@app.route("/plainReviewsAll", methods=['GET', 'POST'])
def endpointRPA():    
    if request.method == 'POST':
        t0 = time.time()
        try:
            plainCDF = reviewsPlain.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            tryCount = request.form['tryCount']
            filtered = fil2.main(plainCDF, "", newName, "", "", "")                
            if (filtered.empty):
                return "No data for those filters."  
            filtered.sort_values(by=['date'], ascending=False, inplace=True)  
            status = list()    
            minStart = 20 * (int(tryCount) - 1)
            maxEnd = 20 * int(tryCount) - 1
            filtered = filtered.iloc[minStart:maxEnd]
            if (filtered.empty):
                return "No more reviews to show. Press the button again to view latest reviews!" 
            for i in range(filtered.shape[0]):
                status.append(str(filtered["stars"].iloc[i]))
                status.append(str(filtered["date"].iloc[i]))
                status.append(str(filtered["text"].iloc[i]))
        except:
            return "Python  error: " + traceback.format_exc().splitlines()[-1]
        return jsonify(status)
    else:
        return "Send a proper POST request!"
    
@app.route("/filter", methods=['GET', 'POST'])
def endpointFilter():    
    if request.method == 'POST':
        try:
            global lastDF
            newName = request.form['newName']  
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            granulation = request.form['granulation']            
            if str(lastDF["new_name"].iloc[0]) == newName:
                currentDF = lastDF.copy()
                filtered = fil2.main(currentDF, "", "", "", dateLow, dateUp)
                result = woc3.main(filtered, granulation)
                if (filtered.empty):
                    return "No data for those filters."
            else:
                currentDF = allDataLem.copy()
                filtered = fil2.main(currentDF, "", newName, "", dateLow, dateUp)
                result = woc3.main(filtered, granulation)
                if (filtered.empty):
                    return "No data for those filters."
                lastDF = filtered.copy()
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return jsonify(filtered.to_json(orient='records'), result.to_json(orient='records'))
    else:
        return "Send a proper POST request!"

@app.route("/monthView")
def endpointMV():    
    return  render_template("/SingleYear.html")
    
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2020 16:25:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 16:25:35] "GET /static/stylesheets/index.css HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 16:25:35] "GET /static/stylesheets/PlovdivSans.otf HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 16:25:57] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 16:25:58] "POST /restaurantsNames HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:31:14] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:31:14] "POST /restaurantsNames HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:33:03] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:33:04] "POST /restaurantsNames HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:34:42] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:34:42] "POST /restaurantsNames HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:36:02] "GET /AllSample2 HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 17:36:02] "POST /

In [3]:
def edno():

    t0 = time.time()
    client = MongoClient()
    db = client.test_database
    collection = db.test_collection

    reviewsMongo = db.reviews_mini
    restaurantsMongo = db.restaurants_change
    restaurantsMongoA = db.restaurants_change_a
    restaurantsMongoB = db.restaurants_change_b
    reviews_plainMongo = db.reviews_plain
    
    allDataLem = pd.DataFrame(list(reviewsMongo.find({})))
    t1 = time.time()
    print("Database converted in: " + str((t1-t0)/60))
    del allDataLem["_id"]
    allDataLem = allDataLem.astype({'stars': np.int8})

#edno() 

#t2 = time.time()
#print("allDataLem done in: " + str((t2-t0)/60))

    
#def dve():
    
    #restaurantNames = pd.DataFrame(list(restaurantsMongo.find({})))
    #del restaurantNames["_id"]

    restaurantNamesA = pd.DataFrame(list(restaurantsMongoA.find({})))
    del restaurantNamesA["_id"]
    #print(restaurantNamesA)
    
    restaurantNamesB = pd.DataFrame(list(restaurantsMongoB.find({})))
    del restaurantNamesB["_id"]

#def dveipolovina():
    
    #restaurantNames = restaurantNames.to_json(orient='records')
    restaurantNamesA = restaurantNamesA.to_json(orient='records')
    restaurantNamesB = restaurantNamesB.to_json(orient='records')

#dve()
#dveipolovina()
    
#t3 = time.time()
#print("Restaurants done in: " + str((t3-t2)/60))

#def tri():
    reviewsPlain = pd.DataFrame(list(reviews_plainMongo.find({})))
    del reviewsPlain["_id"]
    reviewsPlain = reviewsPlain.astype({'stars': np.int8})
    
#tri()

#t4 = time.time()
#print("Just reviews in: " + str((t4-t3)/60))

#print("All done in: " + str((t4-t0)/60))

mem_usage = memory_usage(edno)
print('Memory usage (in chunks of .1 seconds): %s' % mem_usage)
print('Maximum memory usage for edno: %s' % max(mem_usage))

#mem_usage = memory_usage(dve)
#print('Memory usage (in chunks of .1 seconds): %s' % mem_usage)
#print('Maximum memory usage for dve: %s' % max(mem_usage))
#mem_usage = memory_usage(dveipolovina)
#print('Memory usage (in chunks of .1 seconds): %s' % mem_usage)
#print('Maximum memory usage for dveipolovina: %s' % max(mem_usage))
#mem_usage = memory_usage(tri)
#print('Memory usage (in chunks of .1 seconds): %s' % mem_usage)
#print('Maximum memory usage for tri: %s' % max(mem_usage))

Database converted in: 1.2151313583056131
Memory usage (in chunks of .1 seconds): [97.484375, 97.51953125, 132.98046875, 169.5390625, 195.76171875, 230.87109375, 268.76171875, 289.75, 321.1015625, 331.6171875, 351.2734375, 396.484375, 423.98828125, 458.1171875, 485.28125, 491.02734375, 505.21875, 554.32421875, 586.73046875, 617.015625, 639.9609375, 644.79296875, 668.5703125, 709.328125, 760.98046875, 773.88671875, 823.01171875, 832.203125, 832.203125, 848.7421875, 891.6796875, 912.953125, 955.8671875, 1003.4921875, 1019.42578125, 1046.625, 1046.625, 1046.625, 1075.0703125, 1113.31640625, 1164.62109375, 1181.32421875, 1232.8046875, 1260.8125, 1302.390625, 1332.078125, 1332.078125, 1332.078125, 1317.28125, 1356.234375, 1409.11328125, 1424.4453125, 1477.9296875, 1504.3359375, 1542.80078125, 1562.6328125, 1602.69140625, 1655.06640625, 1663.58203125, 1663.58203125, 1663.58203125, 1663.58203125, 1678.84375, 1731.9609375, 1745.984375, 1794.3359375, 1816.11328125, 1863.8359375, 1889.65234375, 

In [16]:
allDataLem["lemmas"][5]

['\ufeff1',
 'go',
 'view',
 'bellagio',
 'fountain',
 'really',
 'good',
 'get',
 'table',
 'outside',
 'stand',
 'heat',
 'food',
 '3',
 'time',
 'food',
 'way',
 'price',
 'quality',
 'egg',
 'benedict',
 'last',
 'time',
 'muffin',
 'flat',
 'little',
 'sauce',
 'bloody',
 'mary',
 'okay',
 'service',
 'good',
 'want',
 'meal',
 'view',
 'fountain',
 'try',
 'somewhere',
 'else']

In [17]:
reviewsPlain['text'][5]

'First, you go here for the view of the Bellagio fountains. Which is really good if you get a table outside and can stand the heat.\r\r\r\n\r\r\r\nAs for the food, been here 3 times and the food is way over priced for the quality.  Had eggs benedict last time and the muffins were flat and there was very little sauce. Bloody mary was just ok. \r\r\r\n\r\r\r\nService was good, but if you want a meal with your view of the fountains, try somewhere else.'

In [34]:
restaurantsMongoA = db.restaurants_change_a
restaurantsMongoB = db.restaurants_change_b

restaurantsMongoA.insert_many(restaurantNames[0].to_dict('records'))
restaurantsMongoB.insert_many(restaurantNames[1].to_dict('records'))

In [17]:
reviewsPlainProba = reviewsPlain.copy()
reviewsPlainProba

,stars,date,text,new_name
0,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi"" - 89109"
1,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi"" - 89109"
2,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi"" - 89109"
3,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi"" - 89109"
4,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi"" - 89109"
...,...,...,...,...
3346019,1,2017-08-18,Bar is absolute trash. Came in spend over 150 ...,"""Timbers Rancho"" - 89130"
3346020,4,2017-10-13,Nice place! I am a non-smoker and although a f...,"""Timbers Rancho"" - 89130"
3346021,5,2009-05-22,Are you kidding me?? 1 star? \r\r\r\n\r\r\r\nO...,"""Budweiser Racing Track Bar & Grill"" - 89119"
3346022,2,2013-05-14,$14.35 for a beer and a coke. \r\r\r\n\r\r\r\n...,"""Budweiser Racing Track Bar & Grill"" - 89119"


In [21]:
stringa1 = """"McDonald'"""
stringa2 = """s"""
stringa3 = stringa1 + stringa2
print(stringa3)
zaProbi = fil2.main(reviewsPlainProba, "", stringa3 + '" - M4C 1H9', "", '2016-08-01', '2016-08-31')
zaProbi

"McDonald's


,stars,date,text,new_name
3339556,1,2016-08-31,Wrote them up on their horrible customer servi...,"""McDonald's"" - M4C 1H9"


In [54]:
t0 = time.time()
reviewsPlainProba["text"].str.contains("good")

indexNames = reviewsPlainProba[reviewsPlainProba["text"].str.contains("Having", regex=False)==False].index
reviewsPlainProba.drop(indexNames , inplace=True)  

t1 = time.time()
print("Clocks at: " + str(t1-t0))
reviewsPlainProba

Clocks at: 7.203769207000732


,stars,date,text,new_name
3,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi"" - 89109"
49,4,2010-12-29,"What is French for ""OMG I would totally go bac...","""Mon Ami Gabi"" - 89109"
208,5,2015-06-15,"We enjoyed dinner here prior to seeing O, and ...","""Mon Ami Gabi"" - 89109"
248,5,2016-09-05,Having a great time hanging out at the patio b...,"""Mon Ami Gabi"" - 89109"
393,3,2014-02-28,"Having Mon Ami Gabi bookmarked for awhile, I f...","""Mon Ami Gabi"" - 89109"
...,...,...,...,...
3344906,2,2010-07-03,Eek me thinks not is right! \r\r\r\n\r\r\r\n...,"""La Belle Province"" - H1W 2G2"
3345002,4,2011-06-16,I had dinner with two friends here the other n...,"""DiMartino's Italian Restaurant"" - 89123"
3345070,1,2015-12-29,Having been a Starbucks Barista myself that wo...,"""Starbucks"" - M8Y 0B6"
3345185,4,2014-08-19,"A newcomer to the Spring Mountain scene, I'm g...","""A & K Chinese Restaurant"" - 89146"


In [5]:
t0 = time.time()
reviewsPlainProba = reviewsPlainProba[reviewsPlainProba['text'].str.contains("Having")]
t1 = time.time()
print("Clocks at: " + str(t1-t0))
reviewsPlainProba

NameError: name 'reviewsPlainProba' is not defined